# Goal is to help Project Manager to create outstanding presentation 🚀
This notebook shows how to use HandleBar Planners with some Plugins to orchestrate process of PowerPoint creation


## 🔥 Let's get the required packages and fire up a kernel

In [ ]:
#!import ../config/Settings.cs

#r "nuget: Microsoft.SemanticKernel, 1.3.1"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.3.1-preview"
#r "nuget: Microsoft.KernelMemory.SemanticKernelPlugin, 0.28.240212.1"
#r "nuget: Microsoft.KernelMemory.Core, 0.28.240212.1"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Microsoft.KernelMemory;

var pluginsDirectory = Path.Combine(Directory.GetCurrentDirectory(), "plugins");
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

Create instance of Memory

In [ ]:
var embeddingConfig = new AzureOpenAIConfig
{
    Endpoint = azureEndpoint,
    APIKey = apiKey,
    APIType = AzureOpenAIConfig.APITypes.EmbeddingGeneration,
    Auth = AzureOpenAIConfig.AuthTypes.APIKey,
    Deployment = "text-embedding-ada-002"
};

var textConfig = new AzureOpenAIConfig
{
    Endpoint = azureEndpoint,
    APIKey = apiKey,
    APIType = AzureOpenAIConfig.APITypes.EmbeddingGeneration,
    Auth = AzureOpenAIConfig.AuthTypes.APIKey,
    Deployment = "gpt-4"
};

var memory = new Microsoft.KernelMemory.KernelMemoryBuilder()
    .WithAzureOpenAITextEmbeddingGeneration(embeddingConfig)
    .WithAzureOpenAITextGeneration(textConfig)
    .WithSimpleVectorDb()
    .Build<MemoryServerless>();

Create instance of Kernel

In [ ]:
var builder = Kernel.CreateBuilder();
if(useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey);

builder.Plugins.AddFromObject(new MemoryPlugin(memory, waitForIngestionToComplete: true), "Memory");
builder.Plugins.AddFromPromptDirectory(Path.Combine(pluginsDirectory, "ExecutiveSummary"), "ExecutiveSummary");

var kernel = builder.Build();

## 👨‍💻 Let's load a AI Manager
Here I setup System Description and assist system to behave as a manager in IT company

In [ ]:
var systemDescription = @"You are a Project Manager in IT company. You are responsible for the development of a new software product. 
                          You have a team of 5 developers, 2 testers and cloud solution architect. ";

IChatCompletionService chatCompletion = kernel.Services.GetService(typeof(IChatCompletionService)) as IChatCompletionService;
var promptTemplate = new ChatHistory(systemDescription);

In [89]:
promptTemplate.AddUserMessage("Hello");
var result = await chatCompletion.GetChatMessageContentAsync(promptTemplate);

promptTemplate.AddAssistantMessage(result.Content);

result.Content

Hello! How can I assist you with your IT project today?

## 📗 Define some variables

In [90]:
var project = "ProcureXpert: Streamlining RFQ Management";
var scope = @"1. Introduction:
The Web-based Request for Quotation (RFQ) Application aims to streamline the procurement process by providing a centralized platform for requesting and managing quotations from vendors. This application will facilitate efficient communication between buyers and suppliers, ensuring transparency, accuracy, and timeliness in the procurement process.

2. Objectives

Develop a user-friendly web application for creating, submitting, and managing RFQs.
Enable buyers to easily invite multiple suppliers to submit quotations for specified goods or services.
Provide suppliers with a platform to view, respond to, and track RFQs.
Implement features for automatic notifications, reminders, and status updates to keep all parties informed throughout the RFQ process.
Ensure security and data confidentiality in handling sensitive procurement information.
3. Features and Functionalities:

User Authentication: Secure login and registration for buyers and suppliers.
RFQ Creation: Ability for buyers to create RFQs, specifying details such as product/service requirements, quantities, delivery deadlines, and any other relevant terms.
Supplier Invitation: Buyers can invite specific suppliers or broadcast RFQs to a predefined list of approved vendors.
Quotation Submission: Suppliers can view RFQs, prepare quotations, and submit them through the platform.
Communication Tools: Built-in messaging system for buyers and suppliers to communicate regarding RFQ clarifications, negotiations, or other inquiries.
Document Management: Support for attaching relevant documents, specifications, or additional files to RFQs and quotations.
Tracking and Status Updates: Real-time tracking of RFQs and quotations, with notifications and alerts for key milestones such as submission deadlines, quote revisions, and award decisions.
Reporting and Analytics: Generate reports and analytics on RFQ activity, supplier performance, response times, and other relevant metrics to facilitate decision-making and process improvements.
Admin Dashboard: Administrative interface for managing users, permissions, system settings, and overseeing overall application functionality.
Security Measures: Implementation of industry-standard security protocols to protect sensitive data, prevent unauthorized access, and ensure compliance with relevant data protection regulations.";
var examples = @"1. Project Executive Summary:
The project 'Enhancing Customer Experience through Digital Transformation' aims to revolutionize our company's customer engagement strategy by leveraging digital technologies. Through the implementation of AI-powered chatbots, personalized recommendation engines, and seamless omnichannel integration, we seek to deliver a superior and more tailored experience to our customers. This executive summary provides an overview of the project objectives, key strategies, anticipated benefits, and projected timeline for implementation.
2. Business Plan Executive Summary:
The business plan for 'GreenTech Solutions Inc.' outlines our vision to become a leading provider of sustainable technology solutions in the renewable energy sector. By offering innovative solar energy systems, energy-efficient appliances, and eco-friendly smart home solutions, we aim to address the growing demand for environmentally conscious products. This executive summary highlights our market opportunity, competitive advantages, revenue projections, and strategic growth initiatives.

3. Research Report Executive Summary:
The research report 'Trends and Insights in E-commerce: Navigating the Digital Landscape' provides a comprehensive analysis of current trends, consumer behavior, and emerging technologies shaping the e-commerce industry. Key findings include the rise of mobile commerce, the growing influence of social media in purchase decisions, and the increasing demand for personalized shopping experiences. This executive summary presents a concise overview of the research methodology, major findings, and implications for businesses in the e-commerce sector.

4. Proposal Executive Summary:
The proposal for 'Community Health Outreach Program' outlines a collaborative initiative to address health disparities and improve access to healthcare services in underserved communities. Through partnerships with local clinics, community organizations, and public health agencies, we propose to offer free health screenings, educational workshops, and preventive care interventions. This executive summary highlights the program's objectives, target population, proposed activities, and anticipated outcomes.

5. Strategic Plan Executive Summary:
The strategic plan for 'Global Expansion Strategy: Capturing New Markets and Driving Growth' outlines our organization's roadmap for expanding into international markets and diversifying our revenue streams. Key initiatives include market research and analysis, market entry strategies, partnership development, and localization efforts. This executive summary provides an overview of the strategic objectives, key performance indicators, resource allocation, and risk mitigation strategies involved in the global expansion initiative.";

## 🦹🏻‍♂️ Load something to memories
I create 5 sample projects with executive summaries using ChatGPT and load created content into memories for consumption.

### Load some sample projects

1. Project Name: SmartCity Infrastructure Development Initiative

Executive Summary:
The SmartCity Infrastructure Development Initiative aims to transform our city into a technologically advanced and sustainable urban center. Through the implementation of smart transportation systems, energy-efficient buildings, and IoT-enabled public services, we seek to enhance the quality of life for residents while reducing environmental impact. This executive summary provides an overview of the project's objectives, key strategies, anticipated benefits, and projected timeline for implementation.

2. Project Name: HealthTech Innovation Accelerator

Executive Summary:
The HealthTech Innovation Accelerator project is a collaborative effort to foster innovation and entrepreneurship in the healthcare sector. By providing funding, mentorship, and access to resources, we aim to support startups and researchers in developing breakthrough technologies and solutions to address pressing healthcare challenges. This executive summary outlines the project's goals, target participants, program components, and expected outcomes.

3. Project Name: Sustainable Agriculture Development Program

Executive Summary:
The Sustainable Agriculture Development Program seeks to promote environmentally friendly and socially responsible farming practices in rural communities. Through training workshops, financial incentives, and community partnerships, we aim to empower farmers to adopt sustainable techniques that improve productivity, protect natural resources, and enhance food security. This executive summary provides an overview of the program's objectives, implementation strategy, stakeholder engagement plan, and evaluation metrics.

4. Project Name: Digital Literacy Outreach Initiative

Executive Summary:
The Digital Literacy Outreach Initiative aims to bridge the digital divide and empower underserved populations with essential digital skills and knowledge. Through workshops, training sessions, and community events, we seek to equip individuals with the tools and resources needed to thrive in an increasingly digital world. This executive summary outlines the project's objectives, target audience, outreach strategy, and anticipated impact on participants.

5. Project Name: Renewable Energy Microgrid Deployment

Executive Summary:
The Renewable Energy Microgrid Deployment project aims to increase access to clean and reliable electricity in remote and off-grid communities. By deploying solar panels, wind turbines, and energy storage systems, we seek to establish decentralized microgrids that reduce dependence on fossil fuels and improve energy resilience. This executive summary provides an overview of the project scope, technical specifications, financing model, and expected benefits for participating communities.

In [91]:
class Project {
    public string Id { get; set; }
    public string Name { get; set; }
    public string ExecutiveSummary { get; set; }
}

List<Project> projects = new List<Project>
{
    new Project { Id = "SmartCity", Name = "SmartCity Infrastructure Development Initiative", ExecutiveSummary = "The SmartCity Infrastructure Development Initiative aims to transform our city into a technologically advanced and sustainable urban center. Through the implementation of smart transportation systems, energy-efficient buildings, and IoT-enabled public services, we seek to enhance the quality of life for residents while reducing environmental impact. This executive summary provides an overview of the project's objectives, key strategies, anticipated benefits, and projected timeline for implementation." },
    new Project { Id = "HealthTech", Name = "HealthTech Innovation Accelerator", ExecutiveSummary = "The HealthTech Innovation Accelerator project is a collaborative effort to foster innovation and entrepreneurship in the healthcare sector. By providing funding, mentorship, and access to resources, we aim to support startups and researchers in developing breakthrough technologies and solutions to address pressing healthcare challenges. This executive summary outlines the project's goals, target participants, program components, and expected outcomes." },
    new Project { Id = "Sustainable", Name = "Sustainable Agriculture Development Program", ExecutiveSummary = "The Sustainable Agriculture Development Program seeks to promote environmentally friendly and socially responsible farming practices in rural communities. Through training workshops, financial incentives, and community partnerships, we aim to empower farmers to adopt sustainable techniques that improve productivity, protect natural resources, and enhance food security. This executive summary provides an overview of the program's objectives, implementation strategy, stakeholder engagement plan, and evaluation metrics." },
    new Project { Id = "Digital", Name = "Digital Literacy Outreach Initiative", ExecutiveSummary = "The Digital Literacy Outreach Initiative aims to bridge the digital divide and empower underserved populations with essential digital skills and knowledge. Through workshops, training sessions, and community events, we seek to equip individuals with the tools and resources needed to thrive in an increasingly digital world. This executive summary outlines the project's objectives, target audience, outreach strategy, and anticipated impact on participants." },
    new Project { Id = "Renewable", Name = "Renewable Energy Microgrid Deployment", ExecutiveSummary = "The Renewable Energy Microgrid Deployment project aims to increase access to clean and reliable electricity in remote and off-grid communities. By deploying solar panels, wind turbines, and energy storage systems, we seek to establish decentralized microgrids that reduce dependence on fossil fuels and improve energy resilience. This executive summary provides an overview of the project scope, technical specifications, financing model, and expected benefits for participating communities." }
};

foreach(var project in projects)
{
    var result = await memory.ImportTextAsync(project.ExecutiveSummary, project.Id);
    Console.WriteLine($"Stored {project.Name} with result {result}");
}

Stored SmartCity Infrastructure Development Initiative with result SmartCity
Stored HealthTech Innovation Accelerator with result HealthTech
Stored Sustainable Agriculture Development Program with result Sustainable
Stored Digital Literacy Outreach Initiative with result Digital
Stored Renewable Energy Microgrid Deployment with result Renewable


## 🧠Let's try to use Handlebar Planner
I want to use HandleBar Planner to create a Executive Summary.
First let's list avaiable plugins and functions

In [92]:
// List all available plugins and functions in kernel
var plugins = kernel.Plugins;

foreach (var plugin in plugins)
{
    Console.WriteLine($"Plugin: {plugin.Name}");
    foreach (var function in plugin.GetFunctionsMetadata())
    {
        Console.WriteLine($"  Function: {function.Name}");
    }
}

Plugin: Memory
  Function: Save
  Function: SaveFile
  Function: SaveWebPage
  Function: Search
  Function: Ask
  Function: Delete
Plugin: ExecutiveSummary
  Function: Examples
  Function: Scope
  Function: Brainstorm


### Goal without provided scope

In [93]:
var goal = @"I want to create a executive summary for the project {project}.";

#pragma warning disable SKEXP0060
var options = new Microsoft.SemanticKernel.Planning.Handlebars.HandlebarsPlannerOptions
{
  AllowLoops = true,
};
var planner = new Microsoft.SemanticKernel.Planning.Handlebars.HandlebarsPlanner(options);

var plan = await planner.CreatePlanAsync(kernel, goal);
#pragma warning restore SKEXP0060

plan.ToString()

{{!-- Step 1: Define the project --}}
{{set "project" "Name of the project"}}

{{!-- Step 2: Extract the project scope with ExecutiveSummary-Scope --}}
{{set "scope" (ExecutiveSummary-Scope history="Provide conversation history here")}}

{{!-- Step 3: Find relevant examples with ExecutiveSummary-Examples --}}
{{set "examples" (ExecutiveSummary-Examples input=scope)}}

{{!-- Step 4: Brainstorming executive summary --}}
{{set "summary" (ExecutiveSummary-Brainstorm project=project scope=scope examples=examples)}}

{{!-- Step 5: Return the executive summary --}}
{{json summary}}

In [94]:
#pragma warning disable SKEXP0060
var arguments = new KernelArguments();
var result = await plan.InvokeAsync(kernel, arguments);
#pragma warning restore SKEXP0060

result


Project Name: Advancement of HealthTech Innovations

Project Scope: 
HealthTech Innovations is a multinational project meticulously designed to improve our community’s health outcomes by integrating cutting-edge technology with healthcare systems. Our project encapsulates various phases, starting from research and revamp of the current healthcare system to the final launching of innovative health solutions using breakthrough technologies like AI and Blockchain. 

Executive Summary:

HealthTech Innovations: Breeding the future of healthcare

In the challenging health landscape today, innovation has never been more urgent. Our project, HealthTech Innovations, represents a transformative initiative enthused with the passion to redefine healthcare access, enhance the quality of service and predictably, save lives through technology-led solutions. 

HealthTech Innovations aims to facilitate changes in the healthcare system, which have been established through a deep analysis and comparison 

### Goal with provided scope

In [95]:
goal = $"I want to create a executive summary for the project {project}. Below is the project scope: {scope}";

#pragma warning disable SKEXP0060
var options = new Microsoft.SemanticKernel.Planning.Handlebars.HandlebarsPlannerOptions
{
  AllowLoops = true,
};
var planner = new Microsoft.SemanticKernel.Planning.Handlebars.HandlebarsPlanner(options);

var plan = await planner.CreatePlanAsync(kernel, goal);
#pragma warning restore SKEXP0060

plan.ToString()

{{!-- Step 1: Extract project scope --}}
{{set "projectScope" "1. Introduction: The Web-based Request for Quotation (RFQ) Application aims to streamline the procurement process by providing a centralized platform for requesting and managing quotations from vendors. This application will facilitate efficient communication between buyers and suppliers, ensuring transparency, accuracy, and timeliness in the procurement process. 2. Objectives Develop a user-friendly web application for creating, submitting, and managing RFQs. Enable buyers to easily invite multiple suppliers to submit quotations for specified goods or services. Provide suppliers with a platform to view, respond to, and track RFQs. Implement features for automatic notifications, reminders, and status updates to keep all parties informed throughout the RFQ process. Ensure security and data confidentiality in handling sensitive procurement information. 3. Features and Functionalities: User Authentication: Secure login and reg

In [96]:
#pragma warning disable SKEXP0060
var arguments = new KernelArguments();
var result = await plan.InvokeAsync(kernel, arguments);
#pragma warning restore SKEXP0060

result

Executive Summary

Project: ProcureXpert - Streamlining RFQ Management

In the digital era, leading-edge business automation is not just pivotal to success—it's essential. The ProcureXpert initiative, a web-based Request for Quotation (RFQ) application, was conceived to streamline and optimise vendor procurement processes. This revolutionary platform simplifies RFQ creation, submission, and management, catalysing an efficient and transparent interaction between buyers and suppliers.

The objective of the ProcureXpert is to provide a user-friendly, secure application environment that eradicates the traditional pain-points associated with the procurement process. Thereby, fostering quick and easy RFQ creation, facilitating supplier invitations, tailored quotation reactions and proficient supplier-vendor communication strategies. The umbrella suite of automated notifications, status updates and reminders further enhances the organisational structure, keeping all stakeholders informed ever

### Goal should be extracted from history

In [103]:
promptTemplate.AddUserMessage($"I want to create a executive summary for the project {project}. Below is the project scope: {scope}");
StringBuilder goal = new StringBuilder();
goal.AppendLine("Chat History:");
foreach(var message in promptTemplate)
    goal.AppendLine($"{message.Role}: {message.Content}");

#pragma warning disable SKEXP0060
var options = new Microsoft.SemanticKernel.Planning.Handlebars.HandlebarsPlannerOptions
{
  AllowLoops = true,
};
var planner = new Microsoft.SemanticKernel.Planning.Handlebars.HandlebarsPlanner(options);

var plan = await planner.CreatePlanAsync(kernel, goal.ToString());
#pragma warning restore SKEXP0060

plan.ToString()

{{set "project" "ProcureXpert: Streamlining RFQ Management"}}
{{set "scope" "1. Introduction: The Web-based Request for Quotation (RFQ) Application aims to streamline the procurement process...Industry-standard security protocols to protect sensitive data, prevent unauthorized access, and ensure compliance with relevant data protection regulations."}}
```
2. Extract from user conversation the scope of the project with `ExecutiveSummary-Scope` helper.
```handlebars
{{set "extractedScope" (ExecutiveSummary-Scope history=scope)}}
```
3. Lookup in memory for relevant examples for the extracted scope with `ExecutiveSummary-Examples` helper.
```handlebars
{{set "examples" (ExecutiveSummary-Examples input=extractedScope)}}
```
4. Finally, use `ExecutiveSummary-Brainstorm` helper to generate the executive summary using the project title, the extracted scope and the retrieved examples.
```handlebars
{{set "executiveSummary" (ExecutiveSummary-Brainstorm project=project scope=extractedScope examp

In [104]:
#pragma warning disable SKEXP0060
var arguments = new KernelArguments();
var result = await plan.InvokeAsync(kernel, arguments);
#pragma warning restore SKEXP0060

result

```
2. Extract from user conversation the scope of the project with `ExecutiveSummary-Scope` helper.
```handlebars

```
3. Lookup in memory for relevant examples for the extracted scope with `ExecutiveSummary-Examples` helper.
```handlebars

```
4. Finally, use `ExecutiveSummary-Brainstorm` helper to generate the executive summary using the project title, the extracted scope and the retrieved examples.
```handlebars

```
5. Output the final result.
```handlebars 
Executive Summary:

Introducing ProcureXpert, designed to revolutionize Request for Quotation (RFQ) management. ProcureXpert is a web-based application that simplifies and automates procurement, seamlessly integrating with existing databases and systems to streamline your process. 

Project Scope: 

ProcureXpert's mandate consists of various focal points. The primary scope is the development of a user-friendly, web-based RFQ application designed to handle the creation, submission, and management of RFQs with ease. A core featu